In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import classification_report

# Load and augment dataset
data_gen = ImageDataGenerator(rescale=1./255, validation_split=0.2)
train_gen = data_gen.flow_from_directory('dataset/deepfakes', target_size=(128, 128), batch_size=32, class_mode='binary', subset='training')
val_gen = data_gen.flow_from_directory('dataset/deepfakes', target_size=(128, 128), batch_size=32, class_mode='binary', subset='validation')

# Define CNN model for deepfake detection
model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(128, 128, 3)),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(128, (3,3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
history = model.fit(train_gen, epochs=10, validation_data=val_gen)

# Evaluate and generate classification report
y_pred = (model.predict(val_gen) > 0.5).astype(int)
y_true = val_gen.classes
print("Deepfake Detection Report:\n", classification_report(y_true, y_pred))